## Imports

In [1]:
from datasets import load_dataset
from dataclasses import dataclass, field, fields

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    set_seed
)
from trl import SFTTrainer, SFTConfig, TrlParser, setup_chat_format
from peft import LoraConfig

from sklearn.model_selection import train_test_split

import logging
import torch

import os
import json
import random
import numpy as np

/root/miniconda3/envs/LLM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/LLM/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/root/miniconda3/envs/LLM/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value T

## Dataset setting

네이버 지식인 베스트 질문 크롤링: 제목/본문/채택된 답변 본문

In [ ]:
## 원시 데이터 로드
ds = load_dataset("beomi/KoAlpaca-v1.1a")
columns_to_remove = list(ds["train"].features)  ## 전처리 이후 제거할 기존 피쳐

system_prompt = "당신은 다양한 분야의 전문가들이 제공한 지식과 정보를 바탕으로 만들어진 AI 어시스턴트입니다.\
 사용자들의 질문에 대해 정확하고 유용한 답변을 제공하는 것이 당신의 주요 목표입니다. 복잡한 주제에 대해서도\
 이해하기 쉽게 설명할 수 있으며, 필요한 경우 추가 정보나 관련 예시를 제공할 수 있습니다. 항상 객관적이고 중립적인\
 입장을 유지하면서, 최신 정보를 반영하여 답변해 주세요. 사용자의 질문이 불분명한 경우 추가 설명을 요청하고, 당신이\
 확실하지 않은 정보에 대해서는 솔직히 모른다고 말해주세요."

In [ ]:
## 전처리 이전 데이터 예시
ds["train"][0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

In [ ]:
## 전처리 후 저장
train_ds = ds.map(
    lambda sample:
    {"messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": sample["instruction"]},
        {"role": "assistant", "content": sample["output"]}
    ]}
)

train_ds = train_ds.map(remove_columns = columns_to_remove, batched = False)
train_ds = train_ds["train"].train_test_split(test_size = 0.1, seed = 42)

train_ds["train"].to_json("train_dataset.json", orient = "records", force_ascii = False)
train_ds["test"].to_json("test_dataset.json", orient = "records", force_ascii = False)

Creating json from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

Creating json from Arrow format: 100%|██████████| 3/3 [00:00<00:00, 13.43ba/s]


3811926

In [ ]:
## 전처리 이후 데이터 예시
train_ds["train"][0]

{'messages': [{'content': '당신은 다양한 분야의 전문가들이 제공한 지식과 정보를 바탕으로 만들어진 AI 어시스턴트입니다. 사용자들의 질문에 대해 정확하고 유용한 답변을 제공하는 것이 당신의 주요 목표입니다. 복잡한 주제에 대해서도 이해하기 쉽게 설명할 수 있으며, 필요한 경우 추가 정보나 관련 예시를 제공할 수 있습니다. 항상 객관적이고 중립적인 입장을 유지하면서, 최신 정보를 반영하여 답변해 주세요. 사용자의 질문이 불분명한 경우 추가 설명을 요청하고, 당신이 확실하지 않은 정보에 대해서는 솔직히 모른다고 말해주세요.',
   'role': 'system'},
  {'content': '생선찌개를 비린내 없이 끓이는 방법은 무엇인가요?', 'role': 'user'},
  {'content': '생선찌개를 맛있게 끓이는 방법으로 비린내를 없애는 방법이 있습니다. 아래 방법들을 참고해보세요. \n\n- 비린내가 많이 나는 생선찌개에 마지막으로 식초를 넣으면 비린내가 없어집니다. 또한, 생선을 구울 때 껍질에 식초를 바르면 껍질이 벗겨지지 않고 제 모양대로 구울 수 있습니다. \n- 생선찌개를 만들 때 생선이 다 익은 다음 된장을 풀어 넣으면 비린내를 없앨 수 있습니다. \n- 깨끗이 손질한 생선이라도 미처 손질하지 못한 잡티가 붙어 있을 수 있는데, 끓이기 전에 팔팔 끓는 물을 살짝 끼얹으면 비린내도 가시고 국물이 깔끔합니다. \n- 간을 한 국물이 한참 끓으면 그때 생선을 넣습니다. \n\n위 방법을 참고해서 집에서 맛있는 비린내 없는 생선찌개를 만들어 보세요.',
   'role': 'assistant'}]}

## Model: [Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)

* 일단 시험용으로 작은 모델인 3.1-8B instruct를 사용
* nf4 양자화, QLoRA, 
* VRAM 20GB 요구
* huggingface의 trl 라이브러리를 활용

In [ ]:
print(tokenizer.apply_chat_template(train_ds[0]["messages"], tokenize = False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 다양한 분야의 전문가들이 제공한 지식과 정보를 바탕으로 만들어진 AI 어시스턴트입니다. 사용자들의 질문에 대해 정확하고 유용한 답변을 제공하는 것이 당신의 주요 목표입니다. 복잡한 주제에 대해서도 이해하기 쉽게 설명할 수 있으며, 필요한 경우 추가 정보나 관련 예시를 제공할 수 있습니다. 항상 객관적이고 중립적인 입장을 유지하면서, 최신 정보를 반영하여 답변해 주세요. 사용자의 질문이 불분명한 경우 추가 설명을 요청하고, 당신이 확실하지 않은 정보에 대해서는 솔직히 모른다고 말해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

생선찌개를 비린내 없이 끓이는 방법은 무엇인가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

생선찌개를 맛있게 끓이는 방법으로 비린내를 없애는 방법이 있습니다. 아래 방법들을 참고해보세요. 

- 비린내가 많이 나는 생선찌개에 마지막으로 식초를 넣으면 비린내가 없어집니다. 또한, 생선을 구울 때 껍질에 식초를 바르면 껍질이 벗겨지지 않고 제 모양대로 구울 수 있습니다. 
- 생선찌개를 만들 때 생선이 다 익은 다음 된장을 풀어 넣으면 비린내를 없앨 수 있습니다. 
- 깨끗이 손질한 생선이라도 미처 손질하지 못한 잡티가 붙어 있을 수 있는데, 끓이기 전에 팔팔 끓는 물을 살짝 끼얹으면 비린내도 가시고 국물이 깔끔합니다. 
- 간을 한 국물이 한참 끓으면 그때 생선을 넣습니다. 

위 방법을 참고해서 집에서 맛있는 비린내 없는 생선찌개를 만들어 보세요.<|eot_id|>


`-` 파인 튜닝

* 4epoch 80000 step 기준으로 2시간 30분 소요 (GPU가 완전 유휴상태일 경우)

## 모델 테스트: VRAM 16GB 요구

In [2]:
import os
import torch
from random import randint
from datasets import load_dataset
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

`-` 테스트에 사용될 데이터

In [ ]:
test_dataset = load_dataset("json", data_files = os.path.join("", "./test_dataset.json"), split = "train")
random_idx = randint(0, len(test_dataset))
messages = test_dataset[random_idx]["messages"][:2]
answer = test_dataset[random_idx]["messages"][2]

Generating train split: 2116 examples [00:00, 18358.70 examples/s]


In [4]:
messages

[{'content': '당신은 다양한 분야의 전문가들이 제공한 지식과 정보를 바탕으로 만들어진 AI 어시스턴트입니다. 사용자들의 질문에 대해 정확하고 유용한 답변을 제공하는 것이 당신의 주요 목표입니다. 복잡한 주제에 대해서도 이해하기 쉽게 설명할 수 있으며, 필요한 경우 추가 정보나 관련 예시를 제공할 수 있습니다. 항상 객관적이고 중립적인 입장을 유지하면서, 최신 정보를 반영하여 답변해 주세요. 사용자의 질문이 불분명한 경우 추가 설명을 요청하고, 당신이 확실하지 않은 정보에 대해서는 솔직히 모른다고 말해주세요.',
  'role': 'system'},
 {'content': '민트색이 왜 하늘색 계열인 이유는 무엇인가요? 민트색 음식은 색소를 사용해서 만들어진 건가요?',
  'role': 'user'}]

In [16]:
print(answer["content"])

민트색의 개념은 민트 잎에서 온 것이 아니라 입안 청소제를 연상시키는 after dinner mint에서 비롯되었습니다. 초콜릿 코팅과 민트향 버터크림이 섞여 옅은 색을 띄고, 이후 이 색이 민트색이라는 개념으로 자리 잡았습니다. 민트색의 종류는 시간이 지날수록 많아졌습니다. 현재 음식에 사용되는 대부분의 민트는 식용색소를 사용하여 만든 색상입니다.


`-` 기존 모델 결과(파인튜닝 이전)

In [ ]:
origin_model_name = "meta-llama/Llama-3.1-8B-Instruct"

origin_model = AutoModelForCausalLM.from_pretrained(origin_model_name, use_cache = False, device_map = "cuda:0", dtype = torch.bfloat16)
origin_tokenizer = AutoTokenizer.from_pretrained(origin_model_name, use_fast = True)
origin_tokenizer.pad_token = origin_tokenizer.eos_token
origin_tokenizer.padding_side = "left"

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.02it/s]


In [6]:
terminators = [origin_tokenizer.eos_token_id]

input_ids = origin_tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,   ## 생성 시에 맨 뒤 generation 시작하라는 프롬프트 삽입
    return_tensors = "pt").to(origin_model.device)

outputs = origin_model.generate(
    input_ids,
    max_new_tokens = 512,
    eos_token_id = terminators,
    do_sample = True,
    temperature = 0.7,
    top_p = 0.95
)

response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_idx]["messages"][1]["content"]}\n")
print(f"원답변:\n{test_dataset[random_idx]["messages"][2]["content"]}\n")
print(f"생성답변:\n{origin_tokenizer.decode(response, skip_special_tokens = True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


질문:
민트색이 왜 하늘색 계열인 이유는 무엇인가요? 민트색 음식은 색소를 사용해서 만들어진 건가요?

원답변:
민트색의 개념은 민트 잎에서 온 것이 아니라 입안 청소제를 연상시키는 after dinner mint에서 비롯되었습니다. 초콜릿 코팅과 민트향 버터크림이 섞여 옅은 색을 띄고, 이후 이 색이 민트색이라는 개념으로 자리 잡았습니다. 민트색의 종류는 시간이 지날수록 많아졌습니다. 현재 음식에 사용되는 대부분의 민트는 식용색소를 사용하여 만든 색상입니다.

생성답변:
민트색은 하늘색 계열에 속하는 색상입니다. 민트색의 이름이 오리진을 의미하는 '민트'라는 단어에서 유래된 것이며, 이는 민트향이 나는 초록색으로 인해 비롯되었습니다. 그러나 현대적으로는 민트색은 하늘색 계열의 파스텔톤으로 인식되고 있습니다.

민트색 음식은 색소나 색소제가 사용되지 않습니다. 음식에서 민트색을 표현하는 데는 여러 방법이 있습니다. 대표적으로 다음과 같은 방법이 있습니다.

1.  **민트잎** : 민트 잎을 사용하여 음식에 민트향과 색상을 표현할 수 있습니다. 민트 잎은 초록색으로 가시적이기 때문에 민트색 음식을 표현하는 데 사용됩니다.
2.  **민트 추출물** : 민트 잎에서 추출된 민트 추출물을 사용하여 음식에 민트향과 색상을 표현할 수 있습니다.
3.  **식용 색소** : 민트색을 표현하기 위해 식용 색소를 사용할 수 있습니다. 그러나 이러한 경우 일반적으로 천연민트색을 표현하기 위해 사용되지 않습니다.

민트색 음식을 표현하는 데 사용되는 색소는 주로 천연민트색을 표현하기 위해 사용됩니다. 이러한 천연민트색은 일반적으로 식물성 색소, 특히 식물성 색소 중에서 파스텔톤의 색상을 표현하기 위해 사용됩니다.


In [7]:
del origin_model
del origin_tokenizer
torch.cuda.empty_cache()

`-` 파인튜닝 후 모델

In [9]:
SFT_model_name = "./results/all-modules-8-epoch"

SFT_model = AutoModelForCausalLM.from_pretrained(SFT_model_name, use_cache = False, device_map = "cuda:0", dtype = torch.bfloat16)
SFT_tokenizer = AutoTokenizer.from_pretrained(SFT_model_name, use_fast = True)
SFT_tokenizer.pad_token = SFT_tokenizer.eos_token
SFT_tokenizer.pad_token_id = SFT_tokenizer.eos_token_id
SFT_tokenizer.padding_side = "left"

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.07it/s]


In [10]:
terminators = [SFT_tokenizer.eos_token_id]

input_ids = SFT_tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,   ## 생성 시에 맨 뒤 generation 시작하라는 프롬프트 삽입
    return_tensors = "pt").to(SFT_model.device)

outputs = SFT_model.generate(
    input_ids,
    max_new_tokens = 512,       ## prompt 제외 출력 토큰 수. max_length는 prompt 포함
    eos_token_id = terminators,
    do_sample = True,
    temperature = 0.7,
    top_p = 0.95
)

response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_idx]["messages"][1]["content"]}")
print(f"원답변:\n{test_dataset[random_idx]["messages"][2]["content"]}")
print(f"생성답변:\n{SFT_tokenizer.decode(response, skip_special_tokens = True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


질문:
민트색이 왜 하늘색 계열인 이유는 무엇인가요? 민트색 음식은 색소를 사용해서 만들어진 건가요?
원답변:
민트색의 개념은 민트 잎에서 온 것이 아니라 입안 청소제를 연상시키는 after dinner mint에서 비롯되었습니다. 초콜릿 코팅과 민트향 버터크림이 섞여 옅은 색을 띄고, 이후 이 색이 민트색이라는 개념으로 자리 잡았습니다. 민트색의 종류는 시간이 지날수록 많아졌습니다. 현재 음식에 사용되는 대부분의 민트는 식용색소를 사용하여 만든 색상입니다.
생성답변:
민트색은 원래 초록색과 하늘색이 혼합되어 만들어진 색입니다. 이는 민트 잎의 색이 초록색이기 때문입니다. 따라서, 민트색 계열은 보통 하늘색과 연한 초록색을 나타냅니다. 

민트색을 표현할 때에는 색소를 사용할 수도 있지만, 일반적으로는 음식의 원래 색상에 따라 민트색을 표현합니다. 예를 들어, 아이스크림의 경우에는 식물성 염료를 사용하여 초록색과 하늘색을 혼합하여 민트색을 표현합니다. 하지만, 일부 식품에서는 인공 색소를 사용하기도 합니다. 

따라서, 민트색 음식의 경우에는 식물성 염료나 인공 색소를 사용할 수 있으며, 색소 사용 여부에 따라 민트색의 진하기도와 느낌이 달라질 수 있습니다.


In [18]:
del SFT_model
del SFT_tokenizer
torch.cuda.empty_cache()

`-` Best eval: 3500 steps

In [19]:
cp_model_name = "./results/all-modules-8-epoch/checkpoint-3543"

cp_model = AutoModelForCausalLM.from_pretrained(cp_model_name, use_cache = False, device_map = "cuda:0", dtype = torch.bfloat16)
cp_tokenizer = AutoTokenizer.from_pretrained(cp_model_name, use_fast = True)
cp_tokenizer.pad_token = cp_tokenizer.eos_token
cp_tokenizer.pad_token_id = cp_tokenizer.eos_token_id
cp_tokenizer.padding_side = "left"

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.11s/it]


In [20]:
terminators = [cp_tokenizer.eos_token_id]

input_ids = cp_tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,   ## 생성 시에 맨 뒤 generation 시작하라는 프롬프트 삽입
    return_tensors = "pt").to(cp_model.device)

outputs = cp_model.generate(
    input_ids,
    max_new_tokens = 512,       ## prompt 제외 출력 토큰 수. max_length는 prompt 포함
    eos_token_id = terminators,
    do_sample = True,
    temperature = 0.7,
    top_p = 0.95
)

response = outputs[0][input_ids.shape[-1]:]
print(f"질문:\n{test_dataset[random_idx]["messages"][1]["content"]}")
print(f"원답변:\n{test_dataset[random_idx]["messages"][2]["content"]}")
print(f"생성답변:\n{cp_tokenizer.decode(response, skip_special_tokens = True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


질문:
민트색이 왜 하늘색 계열인 이유는 무엇인가요? 민트색 음식은 색소를 사용해서 만들어진 건가요?
원답변:
민트색의 개념은 민트 잎에서 온 것이 아니라 입안 청소제를 연상시키는 after dinner mint에서 비롯되었습니다. 초콜릿 코팅과 민트향 버터크림이 섞여 옅은 색을 띄고, 이후 이 색이 민트색이라는 개념으로 자리 잡았습니다. 민트색의 종류는 시간이 지날수록 많아졌습니다. 현재 음식에 사용되는 대부분의 민트는 식용색소를 사용하여 만든 색상입니다.
생성답변:
민트색은 일반적으로 하늘색 계열의 색을 말합니다. 이는 민트를 뜻하는 영어 단어'mint'가 하늘색을 의미하는'mint green'와 혼동되기 때문입니다. 민트색은 또한 차분하고 쾌적한 이미지를 연상시켜 하얀색 계열의 색상으로 인식됩니다. 하지만 민트색 음식은 일반적으로 색소를 사용해서 만들어지지 않습니다. 대신, 민트색 음식은 주로 '민트'라는 단어를 포함한 이름을 가지고 있습니다. 예를 들어, '민트초코', '민트초코칩' 등이 대표적인 예입니다. 따라서, 민트색 음식은 색소로 만들어지지 않고, 단순히 이름에서 '민트'라는 단어를 포함한 것입니다.


## Inference ALL

In [1]:
import os
import torch
from datasets import load_dataset
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

/root/miniconda3/envs/LLM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_dataset = load_dataset("json", data_files = os.path.join("", "./test_dataset.json"), split = "train")

In [5]:
## 파인튜닝되지 않은 모델의 토크나이저에 적용시켜줄 것.
LLAMA_3_CHAT_TEMPLATE = (
    "{{ bos_token }}"
    "{% for message in messages %}"
        "{% if message['role'] == 'system' %}"
            "{{ '<|start_header_id|>system<|end_header_id|>\n\n' + message['content'] + eos_token }}"
        "{% elif message['role'] == 'user' %}"
            "{{ '<|start_header_id|>user<|end_header_id|>\n\n' + message['content'] +  eos_token }}"
        "{% elif message['role'] == 'assistant' %}"
            "{{ '<|start_header_id|>assistant<|end_header_id|>\n\n'}}"
            "{% generation %}"
            "{{ message['content'] +  eos_token }}"
            "{% endgeneration %}"
        "{% endif %}"
    "{% endfor %}"
    "{%- if add_generation_prompt %}"
    "{{- '<|start_header_id|>assistant<|end_header_id|>\n\n' }}"
    "{%- endif %}"
)

In [6]:
model_name = "./results/all-modules-8-epoch/checkpoint-3543"

model = AutoModelForCausalLM.from_pretrained(model_name, use_cache = False, device_map = "cuda:0", dtype = torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"
tokenizer.chat_template = LLAMA_3_CHAT_TEMPLATE

Loading checkpoint shards: 100%|██████████| 4/4 [00:20<00:00,  5.13s/it]


In [12]:
result = []

for idx in tqdm(range(test_dataset.num_rows // 100)):
    messages = test_dataset[idx]["messages"][:2]

    terminators = [
        tokenizer.eos_token_id,
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True,
        return_tensors = "pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens = 512,
        eos_token_id = terminators,
        do_sample = True,
        temperature = 0.7,
        top_p = 0.95,
        pad_token_id = tokenizer.eos_token_id
    )

    response = outputs[0][input_ids.shape[-1]:]
    question = test_dataset[idx]["messages"][1]["content"]
    answer = test_dataset[idx]["messages"][2]["content"]
    generation = tokenizer.decode(response, skip_special_tokens = True)
    result.append([question, answer, generation])

os.mkdir("inference")

with open("./inference/model_generation_result.txt", "w") as f:
    for line in result:
        f.write(str(line) + "\n")

100%|██████████| 21/21 [02:06<00:00,  6.00s/it]


> `pad_token == eos_token`이지만, 잘 작동. 애초에 `padding_free = True`이고, `assistant_only_loss`로 학습했고, 단일 텍스트 추론에서는 패딩이 사용되지 않음.

In [13]:
data = open("./inference/model_generation_result.txt", "r")
data = [eval(line) for line in data]

In [14]:
result

[['케첩과 마요네즈의 입구 모양은 왜 다른가요? \n보통 케찹과 마요네즈는 한 세트자나요? 그런데 보면 케찹은 원터치 뚜껑에다가 새 케찹은 뚜껑을 돌려서 열면 얇은 알미늄 같은 막도 이꾸또 케찹이 나오는 구멍이 동그랗자나요? 근데 마요네즈는 돌리는 뚜껑에다가 처음에 새로 사도 알미늄 같은 막두 엄꾸나오는 구멍 모양도 별 모양인데 왜 그런가요?',
  '케첩과 마요네즈의 입구 모양이 다른 것은 끈적이는 정도 즉, 물질의 점성(Viscosity) 차이 때문입니다. 마요네즈는 케첩보다 점성이 높기 때문에 돌리는 뚜껑에 별 모양의 입구를 만들어도 내부 액체의 모양이 유지됩니다. 하지만 케첩은 끈적거림이 적기 때문에 별 모양의 입구를 만들어도 케첩 내부의 액체가 곧 둥글게 뭉쳐지기 때문에 케첩의 뚜껑은 동그란 구멍입니다. 따라서 입구 모양은 물질의 점성(Viscosity) 차이 때문입니다.',
  '케첩과 마요네즈는 어떤 이유로 입구 모양이 다르게 나온 것일까요? \n\n케첩과 마요네즈는 각각의 특성에 따라 입구 모양이 다르게 나온 것입니다. 케첩은 고무유체의 특성 때문에 입구 모양이 원형이 아닌, 알미늄 막처럼 얇은 막이 이꼴이 되어야 합니다. 고무유체는 원형에서 원형으로 움직이기 때문에, 원형이 아닌 입구 모양이면 고무유체가 계속해서 원형으로 돌아다니기 때문에 고무유체가 입구를 지나갈 때 문제가 생길 수 있습니다. \n\n마요네즈는 고체유체의 특성 때문에 입구 모양이 별 모양으로 나온 것입니다. 고체유체는 입구 모양이 둥글면 굳어버리기 때문에, 고체유체가 통과하기 쉽도록 별 모양의 입구 모양이 나온 것입니다. \n\n따라서, 케첩은 고무유체를 잘 통과시키기 위해 원형이 아닌 알미늄 막이 이꼴이 된 입구 모양을 가지고 있으며, 마요네즈는 고체유체를 잘 통과시키기 위해 별 모양의 입구 모양을 가지고 있습니다.'],
 ['꽃게를 톱밥에 담는 이유는 무엇일까요?',
  '꽃게를 톱밥에 넣어 운반하는 이유는 꽃게를 동면 상태에 빠뜨리기 위함입니다. 꽃게는 낮에는 잠을 자고

In [15]:
data

[['케첩과 마요네즈의 입구 모양은 왜 다른가요? \n보통 케찹과 마요네즈는 한 세트자나요? 그런데 보면 케찹은 원터치 뚜껑에다가 새 케찹은 뚜껑을 돌려서 열면 얇은 알미늄 같은 막도 이꾸또 케찹이 나오는 구멍이 동그랗자나요? 근데 마요네즈는 돌리는 뚜껑에다가 처음에 새로 사도 알미늄 같은 막두 엄꾸나오는 구멍 모양도 별 모양인데 왜 그런가요?',
  '케첩과 마요네즈의 입구 모양이 다른 것은 끈적이는 정도 즉, 물질의 점성(Viscosity) 차이 때문입니다. 마요네즈는 케첩보다 점성이 높기 때문에 돌리는 뚜껑에 별 모양의 입구를 만들어도 내부 액체의 모양이 유지됩니다. 하지만 케첩은 끈적거림이 적기 때문에 별 모양의 입구를 만들어도 케첩 내부의 액체가 곧 둥글게 뭉쳐지기 때문에 케첩의 뚜껑은 동그란 구멍입니다. 따라서 입구 모양은 물질의 점성(Viscosity) 차이 때문입니다.',
  '케첩과 마요네즈는 어떤 이유로 입구 모양이 다르게 나온 것일까요? \n\n케첩과 마요네즈는 각각의 특성에 따라 입구 모양이 다르게 나온 것입니다. 케첩은 고무유체의 특성 때문에 입구 모양이 원형이 아닌, 알미늄 막처럼 얇은 막이 이꼴이 되어야 합니다. 고무유체는 원형에서 원형으로 움직이기 때문에, 원형이 아닌 입구 모양이면 고무유체가 계속해서 원형으로 돌아다니기 때문에 고무유체가 입구를 지나갈 때 문제가 생길 수 있습니다. \n\n마요네즈는 고체유체의 특성 때문에 입구 모양이 별 모양으로 나온 것입니다. 고체유체는 입구 모양이 둥글면 굳어버리기 때문에, 고체유체가 통과하기 쉽도록 별 모양의 입구 모양이 나온 것입니다. \n\n따라서, 케첩은 고무유체를 잘 통과시키기 위해 원형이 아닌 알미늄 막이 이꼴이 된 입구 모양을 가지고 있으며, 마요네즈는 고체유체를 잘 통과시키기 위해 별 모양의 입구 모양을 가지고 있습니다.'],
 ['꽃게를 톱밥에 담는 이유는 무엇일까요?',
  '꽃게를 톱밥에 넣어 운반하는 이유는 꽃게를 동면 상태에 빠뜨리기 위함입니다. 꽃게는 낮에는 잠을 자고